In [5]:
import re
import numpy as np
from fractions import Fraction



def parse_formula(formula):
    """
    Parse a chemical formula into a dictionary of element counts.
    Supports parentheses and multipliers, e.g., Ca(OH)2
    """
    tokens = re.findall(r'([A-Z][a-z]?|\(|\)|\d+)', formula)
    stack = [{}]
    
    i = 0
    while i < len(tokens):
        token = tokens[i]
        if token == "(":
            stack.append({})
        elif token == ")":
            group = stack.pop()
            i += 1
            multiplier = 1
            if i < len(tokens) and tokens[i].isdigit():
                multiplier = int(tokens[i])
            for elem, count in group.items():
                stack[-1][elem] = stack[-1].get(elem, 0) + count * multiplier
        elif token.isdigit():
            prev_elem = tokens[i-1]
            if prev_elem not in "()":
                stack[-1][prev_elem] += int(token) - 1
        else:
            stack[-1][token] = stack[-1].get(token, 0) + 1
        i += 1
    
    return stack[0]

def parse_reaction(equation):
    """
    Parse a chemical reaction into lists of reactants and products.
    """
    left, right = equation.split("->")
    reactants = [f.strip() for f in left.split("+")]
    products = [f.strip() for f in right.split("+")]
    return reactants, products



def balance_equation(equation):
    reactants, products = parse_reaction(equation)
    all_molecules = reactants + products


    elements = set()
    for mol in all_molecules:
        elements.update(parse_formula(mol).keys())
    elements = list(elements)


    matrix = []
    for elem in elements:
        row = []
        for mol in reactants:
            row.append(parse_formula(mol).get(elem, 0))
        for mol in products:
            row.append(-parse_formula(mol).get(elem, 0))
        matrix.append(row)

    A = np.array(matrix, dtype=float)

    u, s, vh = np.linalg.svd(A)
    null_space = np.compress(s <= 1e-10, vh, axis=0)
    coeffs = vh[-1, :] 
    coeffs = coeffs / min(abs(coeffs[np.nonzero(coeffs)]))


    coeffs = [Fraction(c).limit_denominator(1000) for c in coeffs]
    lcm = np.lcm.reduce([f.denominator for f in coeffs])
    coeffs = np.array([int(f * lcm) for f in coeffs])

    gcd = np.gcd.reduce(coeffs)
    coeffs = coeffs // gcd


    reactant_coeffs = coeffs[:len(reactants)]
    product_coeffs = coeffs[len(reactants):]


    left_side = " + ".join(f"{reactant_coeffs[i]} {reactants[i]}" for i in range(len(reactants)))
    right_side = " + ".join(f"{product_coeffs[i]} {products[i]}" for i in range(len(products)))
    return f"{left_side} -> {right_side}"



if __name__ == "__main__":
    print("Chemical Equation Balancer")
    print("Enter an unbalanced reaction like: H2 + O2 -> H2O")
    print("Type 'quit' to exit.")
    
    while True:
        eq = input("\nEnter reaction: ").strip()
        if eq.lower() == "quit":
            break
        try:
            balanced = balance_equation(eq)
            print("Balanced:", balanced)
        except Exception as e:
            print("Error:", e)


Chemical Equation Balancer
Enter an unbalanced reaction like: H2 + O2 -> H2O
Type 'quit' to exit.



Enter reaction:  Fe + O2 -> Fe2O3


Balanced: 4 Fe + 3 O2 -> 2 Fe2O3



Enter reaction:  C2H6 + O2 -> CO2 + H2O


Balanced: 2 C2H6 + 7 O2 -> 4 CO2 + 6 H2O


...


Enter reaction:  C2H5COOH + O2 -> CO2 + H2O


Balanced: 2 C2H5COOH + 7 O2 -> 6 CO2 + 6 H2O



Enter reaction:  C3H7OH + O2 -> CO2 + H2O


Balanced: 2 C3H7OH + 9 O2 -> 6 CO2 + 8 H2O



Enter reaction:  quit
